Steps:


1. Create data model
    - attributes
    - flags
    - groups

**2. Process data model**  
    - index nodes <br>
    - infer nodes <br>
    - **detect networks**
    
3. Explore networks
4. Generate AI reports

# 2. Process data model

## Detect networks

In [ ]:
#%pip install ucimlrepo
import sys
from ucimlrepo import fetch_ucirepo
import polars as pl
sys.path.append("../..")

from toolkit.detect_entity_networks.prepare_model import build_model_with_attributes


computer_hardware  = fetch_ucirepo(id=29)


input_dataframe = pl.DataFrame(computer_hardware.data.original)
pl.Config.set_fmt_str_lengths(200)
input_dataframe.head()

entity_id_column = "VendorName"
columns_to_link = ["ModelName", "MYCT", "MMIN", "MMAX", "CACH", "CHMIN", "CHMAX", "PRP", "ERP"]


main_graph = build_model_with_attributes(input_dataframe, entity_id_column, columns_to_link)

In [ ]:
max_attribute_degree = 10
max_network_size=20
supporting_attribute_types=[]
inferred_links = set()
integrated_flags = pl.DataFrame()
additional_trimmed_attributes=[]

In [ ]:
from toolkit.detect_entity_networks.identify_networks import build_entity_records, build_networks, trim_nodeset


(trimmed_degrees, trimmed_nodes) = trim_nodeset(
    main_graph,
    additional_trimmed_attributes,
    max_attribute_degree,
)

(
    community_nodes,
    entity_to_community,
) = build_networks(
    main_graph,
    trimmed_nodes,
    inferred_links,
    supporting_attribute_types,
    max_network_size,
)

entity_records = build_entity_records(
    community_nodes,
    integrated_flags,
)

### Attributes removed because of high degree

In [ ]:
print("Total: ", len(trimmed_degrees))
pl.DataFrame(
    list(trimmed_degrees),
    schema=["Attribute", "Linked Entities"],
).sort("Linked Entities", descending=True).head()

In [ ]:
comm_count = len(community_nodes)

if comm_count > 0:
    comm_sizes = [
        len(comm)
        for comm in community_nodes
        if len(comm) > 1
    ]
    max_comm_size = max(comm_sizes)
    trimmed_atts = len(trimmed_degrees)
    print(
        f"Networks identified: {comm_count} ({len(comm_sizes)} with multiple entities, maximum {max_comm_size})"
    )